In [4]:
s = 'aaa<aa,aa>, aa, aa<<aa>, <aa,aa>>, aa <aa>'

news = ''
add = True
count = 0

for i in range(len(s)):
    c = s[i]
    if c == '<':
        count += 1
        continue
    if c == '>':
        count -= 1
        continue
    if count == 0:
        news += c

print(news)

aaa, aa, aa, aa 


In [9]:
import pandas as pd

data_dict = {'m': pd.Series(["a", "b", "c", "d"]),
             'two': pd.Series([1, 2, 1, 4])}

df = pd.DataFrame(data_dict)

print(f"DataFrame:\n{df}\n")
print(f"column types:\n{df.dtypes}")

col_one_list = df['m'].tolist()

col_one_arr = df['m'].to_numpy()


DataFrame:
   m  two
0  a    1
1  b    2
2  c    1
3  d    4

column types:
m      object
two     int64
dtype: object


In [11]:
df.columns = ["m", "n"]
df

,m,n
0,a,1
1,b,2
2,c,1
3,d,4


In [ ]:
df[df['two'] == 1]['m'].to_numpy()

In [10]:
from pprint import pprint
from difflib import SequenceMatcher, Differ

# public static double Add(int a, int b)
text1 = '''
{
  return a+b;
}
'''.splitlines(True)

# public static double Add(double a, double b)
text2 = '''
{
  // return sum of two numbers
  return a+b;
}
'''.splitlines(True)

# sim = SequenceMatcher(None, text1, text2).ratio()
# print(sim)

d = Differ()
result = list(d.compare(text1, text2))
pprint(result)

eq_lines = sum(l.startswith(' ') for l in result)

print(eq_lines)

array(['a', 'c'], dtype=object)

In [ ]:
from pprint import pprint
from difflib import SequenceMatcher, Differ

# public static double Add(int a, int b)
text1 = '''
{
  return a+b;
}
'''.splitlines(True)

# public static double Add(double a, double b)
text2 = '''
{
  // return sum of two numbers
  return a+b;
}
'''.splitlines(True)

# sim = SequenceMatcher(None, text1, text2).ratio()
# print(sim)

d = Differ()
result = list(d.compare(text1, text2))
pprint(result)

eq_lines = sum(l.startswith(' ') for l in result)

print(eq_lines)

In [44]:
type_dict = {'int': 'Int32', 'long': 'Int64', 'bool': 'Boolean', 'byte': 'Byte', 'decimal': 'Decimal',
             'string': 'String', 'double': 'Double', 'object': 'Object', 'char': 'Char'}  # float missing
def get_type_outbox(param_type):
    if param_type in type_dict:
        return type_dict[param_type]
    return param_type


def replace_by_token(param_type):
    delims = "<>,[]"
    token = ''
    new_param_type = ''
    found_delim = False
    for i in range(len(param_type)):
        if param_type[i] in delims:
            found_delim = True
            replace = get_type_outbox(token)
            token = ''
            new_param_type = new_param_type + replace + param_type[i]
        else:
            token = token + param_type[i]
    if found_delim is False:
        return get_type_outbox(param_type)
    return new_param_type


def replace_types(param_type):
    for k,v in type_dict.items():
        if k in param_type:
            param_type = param_type.replace(k, v)
    return param_type


def change_method_name_commits(method):
    method = method.replace('::', '.')
    start_parameters = method.rfind('(')
    method_name = method[:start_parameters]
    parameters_text = method[start_parameters:]
    if len(parameters_text) > 2:
        parameters_text = parameters_text[1:-1]  # remove ()
        parameters = parameters_text.split(', ')
        params_types = []
        for param in parameters:
            # remove potential spaces from start and end
            if "=" in param:
                param = param[:param.find("=")]
            if "." in param:
                param = param[param.rfind(".") + 1:]
            param = param.strip(" ")
            p = param.split(' ')
            # System.Uri _ -> Uri
            # out T returnedValue -> T&  or out DateTime currencyDate -> DateTime&
            if " ? " in param:
                # if int ? _ -> Nullable<Int32>
                p_t = replace_by_token(p[0])
                param_type = "Nullable<" + p_t + ">"
                if "[]" in ''.join(p[0:-1]):
                    param_type = param_type + "[]"
            else:
                start = 0
                end = -1
                if (p[0] == "params") or (p[0] == "this") or (p[0] == "out") or (p[0] == "ref"):
                    # if params string [] _ -> String[] or params Func<T,object> [] _ -> Func<T,Object>[]
                    start = 1
                p_t = ''.join(p[start:end])
                param_type = replace_by_token(p_t)
                if (p[0] == "out") or (p[0] == "ref"):
                    param_type = param_type + '&'

            params_types.append(param_type)
        parameters_text = '(' + ','.join(params_types) + ')'

    new_name = method_name + parameters_text
    return new_name


methods = '''
GES_GRT.Authorization.Permissions::PermissionAppService::AddPermission( Permission permission , IReadOnlyList<Permission> allPermissions , List<FlatPermissionWithLevelDto> result , int level)
GES_GRT.Authorization.Roles::RoleAppService::UpdateGrantedPermissionsAsync( Role role , List<string> grantedPermissionNames)
GES_GRT.Authorization.Users::LinkedUserDto::GetShownLoginName( bool multiTenancyEnabled)
GES_GRT.DataExporting.Excel.EpPlus::EpPlusExcelExporterBase::AddHeader( ExcelWorksheet sheet , params string [ ] headerTexts)
GES_GRT.Editions::EditionAppService::GetEditionComboboxItems( int ? selectedEditionId = null)
GES_GRT.WebApi.Controllers::ImportIncomeComponentsController::ParseIncomeComponentRows( Range [ ] parsedClientIncomeComponentData)
GES_GRT.Employees.Exporting::EmployeeListExcelExporter::ExportToXlsmFile( List<EmployeeOutputDto> employeeOutputDtos , DateTime ? start , DateTime ? end , User user)
GES_GRT.DataExporting.Excel.EpPlus::EpPlusExcelExporterBase::AddObjects<T>( ExcelWorksheet sheet , int startRowIndex , IList<T> items , params Func<T,object> [ ] propertySelectors)
GES_GRT.Auditing::AuditLogAppService::AuditLogAppService( IRepository<AuditLog,long> auditLogRepository , IRepository<User,long> userRepository , IAuditLogListExcelExporter auditLogListExcelExporter , INamespaceStripper namespaceStripper)
'''.splitlines()

# for m in methods:
#     print("from: ", m)
#     print("to: ", change_method_name_commits(m))

In [1]:
test = "Int32"
print('int' in test)

False


In [48]:
print(change_method_name_commits("GES_GRT.Authorization.Users::UserEmailer::SendEmailActivationLinkAsync( User<String,String[]> user , params string plainPassword = null)"))
print(change_method_name_commits("GES_GRT.Authorization.Users::UserEmailer::SendEmailActivationLinkAsync( this User user , string plainPassword = null)"))
print(change_method_name_commits("GES_GRT.Authorization.Users::UserEmailer::SendEmailActivationLinkAsync( out User user , string ? [ ] plainPassword = null)"))
print(change_method_name_commits("GES_GRT.Web.Helpers::CustomMultipartFormDataStreamProvider::GetLocalFileName( System . Net . Http . Headers . HttpContentHeaders headers)"))
print(change_method_name_commits("JuliusSweetland.OptiKey.Services::InputService::remove_selectionResultEvent(EventHandler<Tuple<List<Point>,KeyValue,List<String>>> test)"))

GES_GRT.Authorization.Users.UserEmailer.SendEmailActivationLinkAsync(User<String,String[]>,String)
GES_GRT.Authorization.Users.UserEmailer.SendEmailActivationLinkAsync(User,String)
GES_GRT.Authorization.Users.UserEmailer.SendEmailActivationLinkAsync(User&,Nullable<String>[])
GES_GRT.Web.Helpers.CustomMultipartFormDataStreamProvider.GetLocalFileName(HttpContentHeaders)
JuliusSweetland.OptiKey.Services.InputService.remove_selectionResultEvent(EventHandler<Tuple<List<Point>,KeyValue,List<String>>>)


In [50]:
def change_method_name_usage(method):
    if '`' in method:
        idx = method.find('`')
        part = method[idx+1:]
        method = method[:idx] + part[part.find('.'):]
    params_start = method.rfind('(')
    method_name = method[:params_start]
    params_text = method[params_start:]
    if len(params_text) > 2:
        parameters_text = params_text[1:-1]  # remove ()
        parameters = parameters_text.split(',')
        params_types = []
        for param in parameters:
            param = param.strip(" ")
            if param.startswith("params"):
                param = param[len("params"):]
                param = param.strip(" ")
            params_types.append(param)
        params_text = '(' + ','.join(params_types) + ')'

    new_name = method_name + params_text
    return new_name

print(change_method_name_usage("GES_GRT.Web.Views.GES_GRTWebViewPageBase`1.get_AbpSession()"))
print(change_method_name_usage("GES_GRT.Extensions.Enum+&lt;&gt;c`1..cctor()"))
print(change_method_name_usage("GES_GRT.DataExporting.Excel.EpPlus.EpPlusExcelExporterBase.AddHeader(ExcelWorksheet, params String[])"))

GES_GRT.Web.Views.GES_GRTWebViewPageBase.get_AbpSession()
GES_GRT.Extensions.Enum+&lt;&gt;c..cctor()
GES_GRT.DataExporting.Excel.EpPlus.EpPlusExcelExporterBase.AddHeader(ExcelWorksheet,String[])


In [ ]:
import sklearn.metrics as compute_metrics
import matplotlib.pyplot as plt

real_labels_file = "C:/Users/aprodea/work/metrics-tax-compare/classification/methods_labelled.csv"
real_labels_data = pd.read_csv(real_labels_file, sep=';')

metrics_labelled_file = "C:/Users/aprodea/work/metrics-tax-compare/classification/1_all_labels.csv"
metrics_labelled_data = pd.read_csv(metrics_labelled_file, sep=';')

data_combined = pd.merge(left=real_labels_data,
                         right=metrics_labelled_data[['Method', 'CLevel_threshold', 'CLevel_k_means', 'CLevel_em']],
                         on='Method', how='left')


labels = ['high', 'regular', 'low']
conf_matrix = compute_metrics.confusion_matrix(y_true=data_combined['CLevel'], y_pred=data_combined['CLevel_em'], labels=labels)
print(conf_matrix)

disp = compute_metrics.ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=labels)
disp = disp.plot(include_values=True, ax=None, xticks_rotation='horizontal')
plt.show()